In [1]:
from llama_index.llms.huggingface import HuggingFaceLLM


# 初始化大语言模型
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == 'assistant':
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<|assistant|>\n"

    return prompt


def completion_to_prompt(completion):
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"


def get_llm(model_name):
    llm = HuggingFaceLLM(
        model_name=model_name,
        tokenizer_name=model_name,
        context_window=3900,
        max_new_tokens=256,
        model_kwargs={"trust_remote_code": True},
        generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95, "pad_token_id": 151645},
        messages_to_prompt=messages_to_prompt,
        completion_to_prompt=completion_to_prompt,
        device_map="auto",
    )
    return llm

# 初始化三个LLM
base_path = "/home/work/var/data/ssr-share-data/"
# qwen_14b = get_llm(base_path + "Qwen-14B-Chat")
chatglm3 = get_llm(base_path + "chatglm3-6b")
qwen_7b = get_llm(base_path + "Qwen1.5-7B-Chat")

llm_dict = {
    # "Qwen-14B": qwen_14b,
    "ChatGLM3-6B": chatglm3,
    "Qwen1.5-7B": qwen_7b
}


/home/work/anaconda3/envs/llamaindex/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/work/anaconda3/envs/llamaindex/lib/python3.10/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.72s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
from llama_index.core import PromptTemplate

# 基础提示模板
base_prompt = PromptTemplate(
    "请根据以下信息回答问题。如果无法从给定信息中找到答案，请说 '我没有足够的信息来回答这个问题'。\n\n背景信息: {context_str}\n\n问题: {query_str}\n\n回答: "
)

# 思维链提示模板
cot_prompt = PromptTemplate(
    "请根据以下信息回答问题。在给出最终答案之前，请先逐步分析你的思考过程。如果无法从给定信息中找到答案，请解释为什么。\n\n背景信息: {context_str}\n\n问题: {query_str}\n\n思考过程:\n1."
)

# 多角度提示模板
multi_perspective_prompt = PromptTemplate(
    "请根据以下信息从多个角度回答问题。考虑不同的观点和可能性。如果信息不足，请指出并解释为什么。\n\n背景信息: {context_str}\n\n问题: {query_str}\n\n多角度分析:\n1."
)

prompts = {
    "基础": base_prompt,
    "思维链": cot_prompt,
    "多角度": multi_perspective_prompt
}

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Settings,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# 加载本地模型
embed_model_path = "/home/work/var/data/ssr-share-data/m3e-base/"

# 设置全局嵌入模型
Settings.embed_model = HuggingFaceEmbedding(model_name=embed_model_path)

# 加载数据（复用8.2中的数据）
documents = SimpleDirectoryReader('temp_docs').load_data()
vector_index = VectorStoreIndex.from_documents(documents)

def get_enhanced_query_engine(llm, prompt, retriever):
    return RetrieverQueryEngine.from_args(
        retriever,
        llm=llm,
        text_qa_template=prompt,
        node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)]
    )

No sentence-transformers model found with name /home/work/var/data/ssr-share-data/m3e-base/. Creating a new one with MEAN pooling.


In [4]:
import time

# 测试问题（复用8.2中的问题）
test_questions = [
    "哪些国家有著名的自然景观？",
    "有哪些建于公元前的历史遗迹？",
    "世界上最高的现代建筑是什么？",
    "哪些宗教场所最受游客欢迎？",
    "有哪些著名的桥梁地标？"
]

# 使用向量检索器（从8.2中复用）
retriever = vector_index.as_retriever(similarity_top_k=3)

results = {}

for llm_name, llm in llm_dict.items():
    for prompt_name, prompt in prompts.items():
        key = f"{llm_name}-{prompt_name}"
        query_engine = get_enhanced_query_engine(llm, prompt, retriever)
        
        start_time = time.time()
        responses = []
        for question in test_questions:
            response = query_engine.query(question)
            responses.append(str(response))
        end_time = time.time()
        
        results[key] = {
            "responses": responses,
            "time": end_time - start_time
        }

# 打印结果
for key, result in results.items():
    print(f"Model-Prompt: {key}")
    print(f"Total time: {result['time']:.2f} seconds")
    for i, response in enumerate(result['responses']):
        print(f"Q{i+1}: {test_questions[i]}")
        print(f"A{i+1}: {response[:200]}...")  # 打印前200个字符
    print("-" * 50)

/home/work/anaconda3/envs/llamaindex/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/work/anaconda3/envs/llamaindex/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Model-Prompt: ChatGLM3-6B-基础
Total time: 14.19 seconds
Q1: 哪些国家有著名的自然景观？
A1: 以下国家有著名的自然景观：

1. Puerto Rico
2. Botswana
3. Tunisia...
Q2: 有哪些建于公元前的历史遗迹？
A2: 建于公元前的历史遗迹包括：

1. Somalia的宏伟的纪念碑，建于公元-2394年（公元前）
2. Suriname的美丽的历史遗迹，建于公元-1358年（公元前）
3. Germany的令人印象深刻的历史遗迹，建于公元-1477年（公元前）...
Q3: 世界上最高的现代建筑是什么？
A3: 世界上最高的现代建筑是位于阿联酋迪拜的哈利法塔（Burj Khalifa），高度为 828 米（2,717 英尺）。...
Q4: 哪些宗教场所最受游客欢迎？
A4: 这些宗教场所最受游客欢迎：

1. United States Virgin Islands的独特的宗教场所
2. Kyrgyz Republic的现代的宗教场所
3. Pitcairn Islands的美丽的宗教场所...
Q5: 有哪些著名的桥梁地标？
A5: 以下是一些著名的桥梁地标：

1. 中国的港珠澳大桥：位于中国广东省珠江口，全长55公里，是中国最著名的桥梁之一，也是世界上最长的跨海大桥之一。

2. 美国的庞洽特雷恩湖桥：位于美国田纳西州，全长10826米，是美国最著名的桥梁之一，也是世界上最长的湖桥之一。

3. 日本的东京塔桥：位于日本东京，全长386米，是东京最著名的桥梁之一，也是日本最著名的地标之一。

4. 澳大利亚的悉尼歌剧院桥：...
--------------------------------------------------
Model-Prompt: ChatGLM3-6B-思维链
Total time: 22.04 seconds
Q1: 哪些国家有著名的自然景观？
A1: 根据提供的信息，我可以得出以下结论：
- Puerto Rico有著名的自然景观Lake Scottfort。
- Botswana有著名的自然景观Greeneville。
- Tunisia有著名的自然景观Sandrachester。

最终答案: Puerto Rico, Botsw